# DATA SCIENCE USANDO DADOS DO CEAPS  

Nesse trabalho usaremos um dataset coletado do portal do CEAPS (Cota para Exercício da Atividade Parlamentar dos Senadores). Nesse dataset temos informações a respeito dos gastos declarados dos senadores. O objetivo deste tralho é limpar e tratar os dados para, por fim, realizar análises que podem ser úteis para avaliar o gasto dos senadores.

1. Lendo o dataset e concatenado em um único Data Frame
2. Explorando o DATASET
3. DATA Cleaning
    - Limpando a coluna COD_DOCUMENTO
    - Limpando a coluna DATA
    - Limpando a coluna VALOR_REEMBOLSADO
    - Transformando a coluna DOCUMENTO em dados booleanos
    - Convertendo variáveis categóricas em variáveis dummies da coluna TIPO_DESPESA
    - Removendo as colunas que não participarão da análise dos dados

    


## 1. Lendo o dataset e concatenado em um único Data Frame

In [1]:
import pandas as pd

Como o arquivo não abriu imediatamente com o método `read_csv` do pandas, de modo que houve um erro dizendo que o arquivo não poderia ser decodificado, então há a necessidade de determinar o  encoding do arquivo. Para isso, importaremos o `chardet.detect` e prosseguiremos da seguinte maneira:

In [2]:
import chardet
with open('dados/despesa_ceaps_2019.csv', 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result

{'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''}

Agora que sabemos o provável encoding dos dados, isto é, `ISO-8859-1`, vamos usar essa informação no método `read_csv()` do pandas e verificar se todos os arquivos do dataset serão lidos.

Quando estamos trabalhando com dados 'crus' ou RAW data, temos que estar atentos para células que estão preenchidas com valores NaN, ou seja, valores não numéricos que não agregam para a análise posterior dos dados. Para isso usaremos o método `isna()` do pandas, somando todos os números para facilitar a compreensão. Além disso, calcularemos a porcentagem de valores NaN por ano para cada uma das colunas.


In [3]:
df_list=[]
isna_by_year=[]
anos = range(2008,2023)
for ano in anos:
    path='dados/despesa_ceaps_'+str(ano)+'.csv'
    df = pd.read_csv(path,sep=";",skiprows=1,encoding='ISO-8859-1')
    df_list.append(df)
    isna_by_year.append(df.isna().sum().values/df.shape[0])


Após criar um dataframe para que arquivo csv, vamos concatenar todos eles em um único data frame.

In [4]:
df = pd.concat(df_list, ignore_index=True)

## 2. Explorando o DATASET

Agora que temos todo o dataset podemos fazer uma análise exploratória dos dados.

In [5]:
df.head()

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO
0,2008,9,ADA MELLO,"Contratação de consultorias, assessorias, pesq...",NaN,NaN,NaN,NaN,NaN,"12351,52",2.008091e+12
1,2008,9,ADA MELLO,"Locomoção, hospedagem, alimentação, combustíve...",NaN,NaN,NaN,NaN,NaN,"386,6",2.008091e+12
2,2008,10,ADA MELLO,"Contratação de consultorias, assessorias, pesq...",NaN,NaN,NaN,NaN,NaN,"12351,52",2.008101e+12
3,2008,10,ADA MELLO,"Locomoção, hospedagem, alimentação, combustíve...",NaN,NaN,NaN,NaN,NaN,"2610,68",2.008101e+12
4,2008,11,ADA MELLO,"Contratação de consultorias, assessorias, pesq...",NaN,NaN,NaN,NaN,NaN,"12351,52",2.008111e+12


In [6]:
print('O dataframe df possui %d linhas e %d colunas'%(df.shape[0],df.shape[1]))

O dataframe df possui 312597 linhas e 11 colunas


Conseguimos informações interessantes com o método `info()` do DataFrame, como, por exemplo, o Data type e a quantidados de valores não nulos.

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312597 entries, 0 to 312596
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   ANO                312597 non-null  int64  
 1   MES                312597 non-null  int64  
 2   SENADOR            312597 non-null  object 
 3   TIPO_DESPESA       312597 non-null  object 
 4   CNPJ_CPF           309453 non-null  object 
 5   FORNECEDOR         309453 non-null  object 
 6   DOCUMENTO          288099 non-null  object 
 7   DATA               309453 non-null  object 
 8   DETALHAMENTO       177658 non-null  object 
 9   VALOR_REEMBOLSADO  312597 non-null  object 
 10  COD_DOCUMENTO      312595 non-null  float64
dtypes: float64(1), int64(2), object(8)
memory usage: 26.2+ MB


Verificando se há valores NaN

In [8]:
df.isna().sum()

ANO                       0
MES                       0
SENADOR                   0
TIPO_DESPESA              0
CNPJ_CPF               3144
FORNECEDOR             3144
DOCUMENTO             24498
DATA                   3144
DETALHAMENTO         134939
VALOR_REEMBOLSADO         0
COD_DOCUMENTO             2
dtype: int64

Verificando se há valores Null

In [9]:
df.isnull().sum()

ANO                       0
MES                       0
SENADOR                   0
TIPO_DESPESA              0
CNPJ_CPF               3144
FORNECEDOR             3144
DOCUMENTO             24498
DATA                   3144
DETALHAMENTO         134939
VALOR_REEMBOLSADO         0
COD_DOCUMENTO             2
dtype: int64

Agora vamos sesparar uma lista com as colunas que apresentam células com valores não numéricos

In [10]:
missing_data_columns=['DETALHAMENTO','CNPJ_CPF','DOCUMENTO','DATA','COD_DOCUMENTO','FORNECEDOR']

Fração de dados NaN para cada coluna e para cada ano

In [11]:
df_na=pd.DataFrame(isna_by_year, columns=df.columns)
df_na=df_na[missing_data_columns]
df_na.insert(0,'ANOS',anos)
df_na

,ANOS,DETALHAMENTO,CNPJ_CPF,DOCUMENTO,DATA,COD_DOCUMENTO,FORNECEDOR
0,2008,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000
1,2009,1.000000,0.042172,0.129490,0.042172,0.000000,0.042172
2,2010,1.000000,0.000000,0.057856,0.000000,0.000000,0.000000
3,2011,0.801947,0.000000,0.060882,0.000000,0.000000,0.000000
4,2012,0.743198,0.000000,0.072278,0.000000,0.000000,0.000000
5,2013,0.177764,0.000000,0.075231,0.000000,0.000000,0.000000
6,2014,0.001409,0.000000,0.074130,0.000000,0.000000,0.000000
7,2015,0.155550,0.000000,0.161422,0.000000,0.000077,0.000000
8,2016,0.202107,0.000000,0.078041,0.000000,0.000000,0.000000
9,2017,0.227843,0.000000,0.048810,0.000000,0.000000,0.000000


## 3. DATA Cleaning

### Limpando a coluna COD_DOCUMENTO

Na sessão Explorando o Data set vimos que a coluna COD_DOCUMENTO apresentou problemas em apenas duas linhas.

In [12]:
df_tratado=df.copy()

In [13]:
import numpy as np 
na_index_list=np.where(df['COD_DOCUMENTO'].isna())[0]
for i in na_index_list:
    print(df.loc[i])
    print('\n')

ANO                                                               2015
MES                                                                  8
SENADOR                                                 EDUARDO AMORIM
TIPO_DESPESA         Passagens aéreas, aquáticas e terrestres nacio...
CNPJ_CPF                                            13.353.495/0001-84
FORNECEDOR                             PROPAGTUR - Propag Turismo Ltda
DOCUMENTO                                      SEM FATURA";02/08/2015"
DATA                 Companhia Aérea: TAM, Localizador: YXGDSJ. Pas...
DETALHAMENTO                                                    469,53
VALOR_REEMBOLSADO                                              2014675
COD_DOCUMENTO                                                      NaN
Name: 150213, dtype: object


ANO                                                               2015
MES                                                                  8
SENADOR                                        

Isso mostra que, para 2 valores do dataframe, não houve separação em colunas considerando o ';'.

In [14]:
for i in na_index_list:
    documento,data=df['DOCUMENTO'].loc[i].split(';')
    documento=documento.replace('\"','')
    data=data.replace('\"','')

    df_tratado['DOCUMENTO'].loc[i]=documento
    df_tratado['DATA'].loc[i]=data
    df_tratado['DETALHAMENTO'].loc[i]=df['DATA'].loc[i]
    df_tratado['VALOR_REEMBOLSADO'].loc[i]=df['DETALHAMENTO'].loc[i]
    df_tratado['COD_DOCUMENTO'].loc[i]=df['VALOR_REEMBOLSADO'].loc[i]

C:\Users\abelh\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [15]:
df_tratado.loc[na_index_list[1]]

ANO                                                               2015
MES                                                                  8
SENADOR                                                 EDUARDO AMORIM
TIPO_DESPESA         Passagens aéreas, aquáticas e terrestres nacio...
CNPJ_CPF                                            13.353.495/0001-84
FORNECEDOR                             PROPAGTUR - Propag Turismo Ltda
DOCUMENTO                                                 ¨¨SEM FATURA
DATA                                                        17/08/2015
DETALHAMENTO         Companhia Aérea: AVIANCA, Localizador: ZNEU9F....
VALOR_REEMBOLSADO                                               460,45
COD_DOCUMENTO                                                  2016557
Name: 150220, dtype: object

In [16]:
i=1
print(df_tratado.loc[na_index_list[i]])

ANO                                                               2015
MES                                                                  8
SENADOR                                                 EDUARDO AMORIM
TIPO_DESPESA         Passagens aéreas, aquáticas e terrestres nacio...
CNPJ_CPF                                            13.353.495/0001-84
FORNECEDOR                             PROPAGTUR - Propag Turismo Ltda
DOCUMENTO                                                 ¨¨SEM FATURA
DATA                                                        17/08/2015
DETALHAMENTO         Companhia Aérea: AVIANCA, Localizador: ZNEU9F....
VALOR_REEMBOLSADO                                               460,45
COD_DOCUMENTO                                                  2016557
Name: 150220, dtype: object


### Limpando a coluna DATA

Observamos na tabela abaixo que, para a coluna DATA, aproximadamente 1% dos dados são NaN

In [17]:
df_tratado.isna().sum()/df_tratado.shape[0]

ANO                  0.000000
MES                  0.000000
SENADOR              0.000000
TIPO_DESPESA         0.000000
CNPJ_CPF             0.010058
FORNECEDOR           0.010058
DOCUMENTO            0.078369
DATA                 0.010058
DETALHAMENTO         0.431671
VALOR_REEMBOLSADO    0.000000
COD_DOCUMENTO        0.000000
dtype: float64

Mas como esses valores NaN estão distribuidos ao longo dos anos? Vimos em explorando o dataset que temos 100% dos valores cmo NaN em 2008 e  aproximadamente 42% de valores Nan em 2009.

In [19]:
import numpy as np

In [20]:
where_is_na=np.where(df_tratado["DATA"].isna())[0]
new_date=pd.to_datetime(df_tratado.ANO.astype(str) + '/' + df_tratado.MES.astype(str)+'/01')#.dt.date

for i in where_is_na:
    df_tratado["DATA"].loc[i]=new_date[i]

C:\Users\abelh\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Agora vamos converter toda a coluna para data. Quando usamos o argumento `errors='coerce'` na função `to_datetime()`, os valores invalidos serão atribuido como `NaT` no dataframe. O array where_is_na indica quais são índices dos elementos do array que foram atribuidos com `True`.

In [21]:
data_tratando=pd.to_datetime(df_tratado["DATA"],errors='coerce')

In [22]:
print('Existem %d valores NaT'%(data_tratando.isna().sum()),'\n')

Existem 21 valores NaT 



In [23]:
where_is_na=np.where(data_tratando.isna())[0]
for i in where_is_na:
    print(str(i),"MES:",df_tratado['MES'].loc[i],' ANO:',df_tratado['ANO'].loc[i],df_tratado['DATA'].loc[i])

154649 MES: 10  ANO: 2015 06/10/2915
156117 MES: 8  ANO: 2015 26/08/0201
156470 MES: 4  ANO: 2015 24/02/5015
162852 MES: 4  ANO: 2015 22/04/0215
162856 MES: 4  ANO: 2015 23/04/0215
167257 MES: 5  ANO: 2015 08/05/5201
172538 MES: 10  ANO: 2016 20/07/5017
182785 MES: 4  ANO: 2016 02/04/3016
183965 MES: 4  ANO: 2016 04/04/0216
184080 MES: 10  ANO: 2016 31/10/0200
188325 MES: 10  ANO: 2016 31/10/0206
203728 MES: 12  ANO: 2017 06/02/1017
212862 MES: 11  ANO: 2017 30/11/1201
213093 MES: 11  ANO: 2017 09/11/0201
215642 MES: 3  ANO: 2017 02/03/0201
223560 MES: 3  ANO: 2017 13/02/0217
238079 MES: 3  ANO: 2018 09/05/0208
240697 MES: 11  ANO: 2018 16/11/1201
259616 MES: 1  ANO: 2019 31/01/0219
280626 MES: 5  ANO: 2020 05/05/0202
286502 MES: 11  ANO: 2021 29/10/0202


Vimos que exitem alguns erros de digitação para os anos, portanto, vamos corrigir esses erros fazendo substituições pelos valores da coluna "ANO".

In [24]:
corrected_dates=[]
for i in where_is_na:
        d,m,y=df_tratado['DATA'].loc[i].split('/')
        corrected_dates.append(str(d)+'/'+m+'/'+str(df_tratado['ANO'].loc[i]))

In [25]:
for i in range(len(where_is_na)):
        df_tratado['DATA'].loc[where_is_na[i]]=corrected_dates[i]

Agora vamos concatenar as colunas 'ANO' e 'MES' na coluna 'PERIODO'

In [26]:
ano_mes=pd.to_datetime(df_tratado.ANO.astype(str) + '/' + df_tratado.MES.astype(str) ).dt.to_period('M')
df_tratado.insert(0,'PERIODO',ano_mes)

In [27]:
df_tratado['DATA']=pd.to_datetime(df_tratado['DATA'],infer_datetime_format=True).dt.date

In [28]:
df_tratado.head(2)

,PERIODO,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO
0,2008-09,2008,9,ADA MELLO,"Contratação de consultorias, assessorias, pesq...",NaN,NaN,NaN,2008-09-01,NaN,"12351,52",2.00809e+12
1,2008-09,2008,9,ADA MELLO,"Locomoção, hospedagem, alimentação, combustíve...",NaN,NaN,NaN,2008-09-01,NaN,"386,6",2.00809e+12


Agora vamos verificar as correções feitas usando o método `info()`.

In [29]:
df_tratado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312597 entries, 0 to 312596
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype    
---  ------             --------------   -----    
 0   PERIODO            312597 non-null  period[M]
 1   ANO                312597 non-null  int64    
 2   MES                312597 non-null  int64    
 3   SENADOR            312597 non-null  object   
 4   TIPO_DESPESA       312597 non-null  object   
 5   CNPJ_CPF           309453 non-null  object   
 6   FORNECEDOR         309453 non-null  object   
 7   DOCUMENTO          288099 non-null  object   
 8   DATA               312597 non-null  object   
 9   DETALHAMENTO       177658 non-null  object   
 10  VALOR_REEMBOLSADO  312597 non-null  object   
 11  COD_DOCUMENTO      312597 non-null  object   
dtypes: int64(2), object(9), period[M](1)
memory usage: 28.6+ MB


### Limpando a coluna VALOR_REEMBOLSADO

Outra informação importante que tiramos do data frame usando o método `df.info()` é que a coluna VALOR_REEMBOLSADO não está sendo lida como sendo de um tipo numérico, mas sim como sendo do tipo object. Portanto, vamos converter a coluna VALOR_REEMBOLSADO para valor numérico da seguinte maneira:

In [30]:
valor_remb=pd.to_numeric(df_tratado['VALOR_REEMBOLSADO'].str.replace(',','.'), errors='coerce')
where_is_na=np.where(valor_remb.isna())[0]

print("Temos que %d de %d valores não foram convertidos para numero."%(len(where_is_na),df_tratado.shape[0]))

Temos que 1 de 312597 valores não foram convertidos para numero.


In [31]:
print(where_is_na)
df_tratado.loc[where_is_na[0]]

[115962]


PERIODO                                                        2013-02
ANO                                                               2013
MES                                                                  2
SENADOR                                                RICARDO FERRAÇO
TIPO_DESPESA         Passagens aéreas, aquáticas e terrestres nacio...
CNPJ_CPF                                            02.012.862/0001-60
FORNECEDOR                                                         TAM
DOCUMENTO                                                9572462859757
DATA                                                        2013-04-02
DETALHAMENTO         CNPJ: 02.012.862/0001-60 FORNECEDOR: TAM DOCUM...
VALOR_REEMBOLSADO                                          1\r\n675,55
COD_DOCUMENTO                                                   698982
Name: 115962, dtype: object

In [32]:
valor_remb[where_is_na[0]]=675.55
df_tratado['VALOR_REEMBOLSADO']=valor_remb
df_tratado['VALOR_REEMBOLSADO'].describe()

count    312597.000000
mean       1005.278140
std        2561.660456
min       -1500.000000
25%         100.000000
50%         280.000000
75%         994.080000
max      270000.000000
Name: VALOR_REEMBOLSADO, dtype: float64

O método `describe()` revelou que há valores negativos na coluna 'VALOR_REEMBOLSADO'. Portanto vamos investigar esses valores e o que podemos fazer sobre isso.

In [33]:
negativos=np.where(valor_remb<0)[0]
print("A coluna 'VALOR_REEMBOLSADO' apresentou %d valores negativos"%(len(negativos)),'\n')

A coluna 'VALOR_REEMBOLSADO' apresentou 3 valores negativos 



Como exitem poucos valores negativos, vamos olhá-los individualmente.

In [34]:
for i in negativos:
    print(df.loc[i],'\n')

ANO                                                               2015
MES                                                                  2
SENADOR                                                 GLADSON CAMELI
TIPO_DESPESA         Aluguel de imóveis para escritório político, c...
CNPJ_CPF                                            63.600.449/0001-00
FORNECEDOR           Arras Adm. de Bens Imóveis Limpeza e Conservaç...
DOCUMENTO                                                          NaN
DATA                                                        15/01/2015
DETALHAMENTO         Contrato de locação de Imóvel comercial do esc...
VALOR_REEMBOLSADO                                                -1500
COD_DOCUMENTO                                              2.00123e+06
Name: 152315, dtype: object 

ANO                                                               2015
MES                                                                  1
SENADOR                                        

In [35]:
for i in negativos:
    print(df['DETALHAMENTO'].loc[i],'\n')
    

Contrato de locação de Imóvel comercial do escritório de apoio do Senador Gladson Cameli no Estado 

9572100929351 - R$ 1.298,93 (1º bilhete emitido) 9572102224403 - R$ 1.418,93 (1ª remarcação) 9572104023443 - R$ 1.498,93 (2ª remarcação) 9572104373256 - R$ 2.412,03 (3ª remarcação - bilhete utilizado) 

Despesa com Monitoramento de Informação Jornalística em Mídia Eletrônica e Imprensa 



Como não conseguimos inferir um novo número para essas linhas iremos excluí-las.

In [36]:
df_tratado=df_tratado.drop(index=negativos).reset_index()

Agora vamos verificar as correções feitas usando o método `info()`.

In [37]:
df_tratado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312594 entries, 0 to 312593
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype    
---  ------             --------------   -----    
 0   index              312594 non-null  int64    
 1   PERIODO            312594 non-null  period[M]
 2   ANO                312594 non-null  int64    
 3   MES                312594 non-null  int64    
 4   SENADOR            312594 non-null  object   
 5   TIPO_DESPESA       312594 non-null  object   
 6   CNPJ_CPF           309450 non-null  object   
 7   FORNECEDOR         309450 non-null  object   
 8   DOCUMENTO          288097 non-null  object   
 9   DATA               312594 non-null  object   
 10  DETALHAMENTO       177655 non-null  object   
 11  VALOR_REEMBOLSADO  312594 non-null  float64  
 12  COD_DOCUMENTO      312594 non-null  object   
dtypes: float64(1), int64(3), object(8), period[M](1)
memory usage: 31.0+ MB


### Transformando a coluna DOCUMENTO em dados booleanos
Vamos transformar a coluna DOCUMENTO em dados booleanos, de modo que o valor True representa a situação em que o senador incluiu o documento e o False representa que o senador não o incluiu.

In [46]:
df_tratado['DOCUMENTO']=pd.to_numeric(df_tratado['DOCUMENTO'].isna()==False)
df_tratado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312594 entries, 0 to 312593
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype    
---  ------             --------------   -----    
 0   index              312594 non-null  int64    
 1   PERIODO            312594 non-null  period[M]
 2   SENADOR            312594 non-null  object   
 3   DOCUMENTO          312594 non-null  bool     
 4   DATA               312594 non-null  object   
 5   VALOR_REEMBOLSADO  312594 non-null  float64  
 6   DESPESA_0          312594 non-null  uint8    
 7   DESPESA_1          312594 non-null  uint8    
 8   DESPESA_2          312594 non-null  uint8    
 9   DESPESA_3          312594 non-null  uint8    
 10  DESPESA_4          312594 non-null  uint8    
 11  DESPESA_5          312594 non-null  uint8    
 12  DESPESA_6          312594 non-null  uint8    
dtypes: bool(1), float64(1), int64(1), object(2), period[M](1), uint8(7)
memory usage: 14.3+ MB


### Convertendo variáveis categóricas em variáveis dummies da coluna TIPO_DESPESA	

Convertendo variáveis categóricas em variáveis dummies conseguimos gerar novas colunas com informações  sobre os tipos de despesa que podem ser analisadas.

In [39]:
dummies=pd.get_dummies(df_tratado['TIPO_DESPESA'])
col=dummies.columns
dummies=pd.get_dummies(df_tratado['TIPO_DESPESA'])
dummies.head(2)

,"Aluguel de imóveis para escritório político, compreendendo despesas concernentes a eles.","Aquisição de material de consumo para uso no escritório político, inclusive aquisição ou locação de software, despesas postais, aquisição de publicações, locação de móveis e de equipamentos.","Contratação de consultorias, assessorias, pesquisas, trabalhos técnicos e outros serviços de apoio ao exercício do mandato parlamentar",Divulgação da atividade parlamentar,"Locomoção, hospedagem, alimentação, combustíveis e lubrificantes","Passagens aéreas, aquáticas e terrestres nacionais",Serviços de Segurança Privada
0,0,0,1,0,0,0,0
1,0,0,0,0,1,0,0


Como os nomes das colunas estão muito extensos, farei um dicionário para esses nomes da seguinte maneira:
 
* **DESPESA_0**: Aluguel de imóveis para escritório político, compreendendo despesas concernentes a eles.
* **DESPESA_1**: Aquisição de material de consumo para uso no escritório político, inclusive aquisição ou locação de software, despesas postais, aquisição de publicações, locação de móveis e de equipamentos. 
* **DESPESA_2**: Contratação de consultorias, assessorias, pesquisas, trabalhos técnicos e outros serviços de apoio ao exercício do mandato parlamentar
* **DESPESA_3**: Divulgação da atividade parlamentar
* **DESPESA_4**: Locomoção, hospedagem, alimentação, combustíveis e lubrificantes
* **DESPESA_5**: Passagens aéreas, aquáticas e terrestres nacionais
* **DESPESA_6**: Serviços de Segurança Privada

In [40]:
dict_col={}
for i in range(len(col)):
    dict_col[col[i]]='DESPESA_'+str(i)
dummies=dummies.rename(columns=dict_col)
dummies.head(2)

,DESPESA_0,DESPESA_1,DESPESA_2,DESPESA_3,DESPESA_4,DESPESA_5,DESPESA_6
0,0,0,1,0,0,0,0
1,0,0,0,0,1,0,0


In [41]:
df_tratado=pd.concat([df_tratado,dummies],axis=1)
df_tratado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312594 entries, 0 to 312593
Data columns (total 20 columns):
 #   Column             Non-Null Count   Dtype    
---  ------             --------------   -----    
 0   index              312594 non-null  int64    
 1   PERIODO            312594 non-null  period[M]
 2   ANO                312594 non-null  int64    
 3   MES                312594 non-null  int64    
 4   SENADOR            312594 non-null  object   
 5   TIPO_DESPESA       312594 non-null  object   
 6   CNPJ_CPF           309450 non-null  object   
 7   FORNECEDOR         309450 non-null  object   
 8   DOCUMENTO          312594 non-null  bool     
 9   DATA               312594 non-null  object   
 10  DETALHAMENTO       177655 non-null  object   
 11  VALOR_REEMBOLSADO  312594 non-null  float64  
 12  COD_DOCUMENTO      312594 non-null  object   
 13  DESPESA_0          312594 non-null  uint8    
 14  DESPESA_1          312594 non-null  uint8    
 15  DESPESA_2        

### Removendo as colunas que não participarão da análise dos dados
* **'ANO' e 'MES'**
    - As colunas 'ANO' e 'MES' foram concatenadas na colunas 'PERIODO'

* **'DETALHAMENTO'**
    - A coluna 'DETALHAMENTO' te muitos valores ausentes e não será usada nas análises.
    
* **'COD_DOCUMENTO'**
    - A coluna 'COD_DOCUMENTO' possuem informação que não serão úteis nas análises.
    
* **'CNPJ_CPF' e 'FORNECEDOR'**
    - Como vimos, quando estavamos explorando os dados, as colunas 'CNPJ_CPF', 'FORNECEDOR' e 'DATA' não apareceram apenas em 2008 e em parte de 2009, de modo que não seria interessante transformar os dados 'CNPJ_CPF' e 'FORNECEDOR' em dados booleanos. 
 
* **'TIPO_DESPESA'**
    - A coluna 'TIPO_DESPESA' não é mais necessária pois foi transformada em colunas com variáveis dummy.

In [42]:
remove_columns=['ANO','MES','DETALHAMENTO','COD_DOCUMENTO','CNPJ_CPF','FORNECEDOR','TIPO_DESPESA']
df_tratado=df_tratado.drop(remove_columns,axis=1)

In [43]:
df_tratado.head(2)

,index,PERIODO,SENADOR,DOCUMENTO,DATA,VALOR_REEMBOLSADO,DESPESA_0,DESPESA_1,DESPESA_2,DESPESA_3,DESPESA_4,DESPESA_5,DESPESA_6
0,0,2008-09,ADA MELLO,False,2008-09-01,12351.52,0,0,1,0,0,0,0
1,1,2008-09,ADA MELLO,False,2008-09-01,386.60,0,0,0,0,1,0,0


In [44]:
df_tratado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312594 entries, 0 to 312593
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype    
---  ------             --------------   -----    
 0   index              312594 non-null  int64    
 1   PERIODO            312594 non-null  period[M]
 2   SENADOR            312594 non-null  object   
 3   DOCUMENTO          312594 non-null  bool     
 4   DATA               312594 non-null  object   
 5   VALOR_REEMBOLSADO  312594 non-null  float64  
 6   DESPESA_0          312594 non-null  uint8    
 7   DESPESA_1          312594 non-null  uint8    
 8   DESPESA_2          312594 non-null  uint8    
 9   DESPESA_3          312594 non-null  uint8    
 10  DESPESA_4          312594 non-null  uint8    
 11  DESPESA_5          312594 non-null  uint8    
 12  DESPESA_6          312594 non-null  uint8    
dtypes: bool(1), float64(1), int64(1), object(2), period[M](1), uint8(7)
memory usage: 14.3+ MB


Agora temos nossos dados limpos e prontos para análises!

## Análise dos dados

In [45]:
df_tratado_senador=df_tratado.drop('index',axis=1).groupby('SENADOR').sum()#describe().loc[:,(slice(None),'max')]
df_tratado_senador.head()

,DOCUMENTO,VALOR_REEMBOLSADO,DESPESA_0,DESPESA_1,DESPESA_2,DESPESA_3,DESPESA_4,DESPESA_5,DESPESA_6
SENADOR,,,,,,,,,
ACIR GURGACZ,2273,4412981.34,415.0,131.0,35.0,406.0,645.0,694.0,0.0
ADA MELLO,0,59962.46,0.0,0.0,4.0,0.0,4.0,0.0,0.0
ADELMIR SANTANA,37,536786.76,4.0,1.0,29.0,3.0,27.0,0.0,0.0
AIRTON SANDOVAL,562,560015.23,200.0,37.0,17.0,0.0,279.0,94.0,0.0
ALESSANDRO VIEIRA,845,1439178.28,168.0,53.0,182.0,1.0,236.0,208.0,0.0
